In [1]:
import numpy as np
import csv
import random
import time

In [4]:
############################## Part 1: Read data  ######################################
## with open("wiser_clean.csv") as csvfile:
with open("wiser_clean.csv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ",")
    input_table = list(reader)

## read 3 prior distr into a single dict.
with open("wiser_prior.csv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ",")
    prior = list(reader)
    
which_prior = 0 # choose from 0,1,2 since we have 3 priors

## convert prior from dict to an array (so that we can perform inner product)
m = len(input_table) # scenarios = 255 for input_table
n = len(input_table[0]) - 1 # test  This should be 78 for input_table
q = np.zeros(m) # prior
for j in range(m):
    q[j] = float(prior[j][str(which_prior)])

In [5]:
exec(open('func_su.py').read()) ## load the basic functions

Next we estimate the ''greedy score'' of a test. Let $E$ be the tests we already selected and $T$ be a test, then $$G(T,E) = \sum_{x}\pi_x \sum_{b:b\sim x} 2^{-m_x} gain(b,x;T,E) = \mathbb{E}_x \big[\mathbb{E}_{b\sim x} [gain(b,x;T,E)]\big],$$
where 
$$gain(b,x;T,E) = \frac{f_{b,x}(E\cup T) - f_{b,x}(E)}{1-f_{b,x}(E)} = \frac{\text{# extra scenarios ruled out by}\ T}{\text{# alive after performing}\ E},$$ if $f_{b,x}(E)<1$, and $0$ otherwise. 

This sum involoves exponentially many terms, so we apply Monto Carlo to estimate it. To be precise, for each $x\in [m]$, we will estimate $\mathbb{E}_{b\sim x} [gain(b,x;T,E)]$ by running Monte Carlo for num_sample epochs, and then sum over $x$. 

In [8]:
# tune-able parameters
#num_sample_MC = 5; num_sample_ECT = 10; num_rep = 10
num_sample_MC = 1; num_sample_ECT = 1; num_rep = 2

recall that if all tests have "low" greedy score, then the future ordering doesn't matter much. Here we set this bar at $10^{-1}$. You should expect the greedy score to be decreasing.

In [ ]:
############################## Part 3: Compute Greedy Permutation  ######################################
skip_stupid_test = 0

for i in range(num_rep): # repeat for num_rep times
    E = [] # tests chosen so far
    not_chosen = list(range(n)) # tests NOT chosen so far
    stop_flag = 0 # when max_gain is too small, we stop and randomly permute the rem- tests

    for t in range(n):
        best_test = random.choice(not_chosen) # randomly choose a test T to start with, if nobody else beats it, then T wins
        if stop_flag == 0: 
            max_gain_so_far = estimate_G(best_test, E, num_sample_MC) # start with the greedy score of this test
            for T in range(n):
                if T in not_chosen: # only choose from unchosen tests
                    temp = estimate_G(T, E, num_sample_MC)
                    if temp > max_gain_so_far: # if T beats the best test
                        max_gain_so_far = temp
                        best_test = T
            print("at round", t, "greedy chooses test", best_test, 
                  "with greedy score", max_gain_so_far)
            
            if max_gain_so_far <= 10**(-1): # if all tests have "low" greedy score, then the future ordering doesn't matter much
                stop_flag = 1
        E.append(best_test)
        not_chosen.remove(best_test)
    print("done!")

## find the average cover time of the greedy permutation.
    ECT = np.zeros(m) # store the ECT of each scenario
    progress_bar_freq = 50 # how often do you want to desplay progress
    for x_hat in range(m):
        temp = 0 # store the sum of cover times (over all epochs)
        for k in range(num_sample_ECT): # Monto Carlo the ECT of x_hat for num_sample_ECT times and take average
            outcome = gen_b(x_hat) # generate outcome vector
            temp += compute_cover_time(E, outcome, skip_stupid_test) # E = greedy perm computed above

        ECT[x_hat] = temp/num_sample_ECT

    print("avg cover time of greedy perm:", np.matmul(q, ECT)) # q = prior distr

at round 0 greedy chooses test 52 with greedy score 0.4850750172470581
